site: https://mleue.com/posts/yield-to-async-await/  
In generators, the behavior at yield statements is limited to “producing” values  
Upto coroutines, a value is sent in when resuming a generator, a function that can be stopped/resumed at specific points and can both produce/receive a values at these points  
site: https://cafedev.vn/tu-hoc-python-coroutine-trong-python/ 
Việc chuyển đổi giữa các threads theo một lịch trình cụ thể được thực hiện bởi hệ điều hành (hoặc run time environment – môi trường thời gian chạy). Trong khi đối với các coroutine, các lập trình viên hoặc ngôn ngữ lập trình sẽ quyết định khi nào chuyển đổi giữa các coroutines. Các coroutines xử lý đa tác vụ một cách kết hợp bằng cách tạm dừng và trở lại thực thi tiếp tại một điểm/mốc được chỉ định bởi lập trình viên  
site: https://superfastpython.com/python-coroutine/  
Perhaps more correctly, a routine is a program, whereas a subroutine is a function in the program.  
A coroutine is a generalization of a subroutine.The main difference is that it chooses to suspend and resume its execution many times before returning and exiting.  
Generators, also known as semicoroutines, are a subset of coroutines  
A subroutine and a coroutine may represent a “task” in a program.  
A coroutine is more lightweight than a thread, a thread is more lightweight than a process. A coroutine is defined as a function. Processe, like thread, is an object created and managed by the underlying operating system and represented in Python as a `threading.Thread` object or a `multiprocessing.Process` object  
A coroutine can be defined via the “async def” expression.  

In [1]:
import asyncio
import time

In [ ]:
# define a coroutine
async def custom_coro():
    # await another coroutine
    await asyncio.sleep(1)
 
# main coroutine
async def main():
    # execute my custom coroutine
    await custom_coro()
 
# start the coroutine program
asyncio.run(main())

In [ ]:
def coro():
    while True:
        value = yield
        print(f"Got: {value}")

c = coro()
c.send(None)
c.send("Hello")
c.send("World")

In [ ]:
def simple_get_page():
    print("Starting to download page")
    time.sleep(1)
    print("Done downloading page")
    return "<html>Hello</html>"

def simple_read_db():
    print("Starting to retrieve data from db")
    time.sleep(0.5)
    print("Connected to db")
    time.sleep(1)
    print("Done retrieving data from db")
    return "db-data"

def run():
	start = time.time()
	simple_get_page()
	simple_read_db()
	print(f"Time elapsed: {time.time()-start:.3}s")
run()

In [ ]:
import heapq
from collections import deque

def scheduler(coros):
    start = time.time()
    ready = deque(coros)
    sleeping = []
    while True:
        if not ready and not sleeping:
            break
            
        # wait for nearest sleeper,
        # if no coro can be executed immediately right now
        if not ready:
            deadline, coro = heapq.heappop(sleeping)
            if deadline > time.time():
                time.sleep(deadline - time.time())
            ready.append(coro)
            
        try:                
            coro = ready.popleft()
            result = coro.send(None)
            # the special case of a coro that wants to be put to sleep
            if len(result) == 2 and result[0] == "sleep":
                deadline = time.time() + result[1]
                heapq.heappush(sleeping, (deadline, coro))
            else:
                print(f"Got: {result}")
                ready.append(coro)
        except StopIteration:
            pass
    print(f"Time elapsed: {time.time()-start:.3}s")

In [ ]:
def get_page_():
    print("Starting to download page")
    yield ("sleep", 1)
    print("Done downloading page")
    yield "<html>Hello</html>"

def read_db_():
    print("Starting to retrieve data from db")
    yield ("sleep", 0.5)
    print("Connected to db")
    yield ("sleep", 1)
    print("Done retrieving data from db")
    yield "db-data"

def write_db_(data):
    print("Starting to write data to db")
    yield ("sleep", 0.5)
    print("Connected to db")
    yield ("sleep", 1)
    print("Done writing data to db")

def worker():
    for step in get_page_():
        yield step
    page = step
    for step in write_db_(page):
        yield step
# scheduler([get_page(), read_db()])
scheduler([worker()])

In [ ]:
def worker_():
    page = yield from get_page_()
    yield from write_db_(page)
scheduler([worker_(), worker_(), worker_()])

In [ ]:
async def get_page():
    print("Starting to download page")
    await time.sleep(1)
    print("Done downloading page")
    return "<html>Hello</html>"

async def write_db(data):
    print("Starting to write data to db")
    await time.sleep(0.5)
    print("Connected to db")
    await time.sleep(1)
    print("Done writing data to db")

async def worker():
    page = await get_page()
    await write_db(page)
scheduler([worker(), worker(), worker()])

In [ ]:
asyncio.run(asyncio.gather(worker(), worker(), worker()))